In [1]:
import sys
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr

import sys
sys.path.append('src')
import DEAPlotting, SpatialTools

In [2]:
results = "results/nmdb_plots/frequency/"

### Part 1: Create rasterstack of irrigation 

In [ ]:
#list of years to help for-loop iterate through folders
x = range(1987,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('2011_12', '2012_13')]
years.sort()

#list of folders to help with loop
folders = os.listdir(directory)
folders.sort()

# Convert our shapefiles into tiffs, add them to an giant xarray and 
# then export as netcdf so we don't have to keep loading them while we
# work out the animation code.
def convertIrrShpToTiff(shp, year):  
    #open a tif and get transform info
    tif = shp[:77]+shp[77:95]+"_multithreshold_65Thres"+suffix[10:]+".tif"
    ds = xr.open_rasterio(tif).squeeze()
    transform, proj = geotransform(ds, (ds.x, ds.y), epsg=3577)
    rows,cols = ds.shape
    #turn vector in numpy array
    shp_arr = rasterize_vector(shp, cols=cols, rows=rows, geo_transform=transform, projection=proj)
    #convert numpy array inot xarray
    shp_xr = xr.DataArray(shp_arr, coords = [ds.y, ds.x], dims = ['y', 'x'])
    #append xarray to list
    da_list.append(shp_xr)

da_list = []
for year, folder in zip(years, folders): 
    print("\r", "working on year: " + year, end = '')
    convertIrrShpToTiff(directory+folder+"/"+"nmdb_Summer"+ year + suffix+".shp", year)

#generate date ranges to use as coordinates in xrray dataset
dates = pd.date_range(start='1/1/1987', end='1/01/2019', freq='Y')
dates = dates.drop([pd.Timestamp('2011-12-31'), pd.Timestamp('2012-12-31')])
#concatenate all xarrays in list to a single multi-dim xarray with time ('dates') as coords.
da = xr.concat(da_list, dim=dates).rename({'concat_dim':'time'}).rename('Irrigated_Area')
#convert to dataset
ds = da.to_dataset()
#export netcdf
ds.to_netcdf(results + "NMDB_irrigation.nc")

### Part 2: Calculate per pixel frequency

In [54]:
irr_alltime = xr.open_dataset(results+'NMDB_irrigation.nc').astype(bool) #.isel(time=range(0,10)).isel(x=range(26000,28000)).isel(y=range(18000,20000))
# irr_90s = xr.open_dataset(results+'NMDB_irrigation.nc').astype(bool).isel(time=range(0,13))
# irr_00s = xr.open_dataset(results+'NMDB_irrigation.nc').astype(bool).isel(time=range(13,23))
# irr_10s = xr.open_dataset(results+'NMDB_irrigation.nc').astype(bool).isel(time=range(23,31))

In [67]:
frequency_alltime = irr_alltime.Irrigated_Area.sum('time', skipna=True) / len(irr_alltime.Irrigated_Area.time)
frequency_alltime = frequency_alltime.rename('freqency of cropping')

In [ ]:
transform, projection = SpatialTools.geotransform(frequency_alltime, (frequency_alltime.x, frequency_alltime.y), epsg=3577)
width,height = frequency_alltime.shape
SpatialTools.array_to_geotiff(results+'frequency_alltime.tif',
              frequency_alltime.values, geo_transform = transform, 
              projection = projection, 
              nodata_val=np.nan)

In [ ]:
# test.Irrigated_Area.sum('time', skipna=True)

In [83]:
pd.date_range(start='1/1/1987', end='1/01/2019', freq='Y')[31]

Timestamp('2018-12-31 00:00:00', freq='A-DEC')

In [85]:
for i in range(23,31):
    print(i)

23
24
25
26
27
28
29
30
